In [1]:
import numpy as np
import pandas as pd

### pandas 对象具有 resample 方法，这是所有频率转换的工具函数。 resample 具有与 groupby 类似的 API：调用 resample 对数据进行分组，然后再调用聚合函数：

In [2]:
dates = pd.date_range("2022-01-01", periods=100)

In [3]:
ts = pd.Series(np.arange(len(dates)), index=dates)
ts

2022-01-01     0
2022-01-02     1
2022-01-03     2
2022-01-04     3
2022-01-05     4
              ..
2022-04-06    95
2022-04-07    96
2022-04-08    97
2022-04-09    98
2022-04-10    99
Freq: D, Length: 100, dtype: int32

In [4]:
ts.resample("M").max()

2022-01-31    30
2022-02-28    58
2022-03-31    89
2022-04-30    99
Freq: M, dtype: int32

In [5]:
# 对区间进行聚合
ts.resample(rule="M" , kind="period").max()

2022-01    30
2022-02    58
2022-03    89
2022-04    99
Freq: M, dtype: int32

# 1.向下采样：

In [6]:
# 首先生成一些一小时内的频率数据
dates = pd.date_range("2022-01-01", periods=12, freq="H")

In [7]:
ts = pd.Series(np.arange(len(dates)), index=dates)
ts

2022-01-01 00:00:00     0
2022-01-01 01:00:00     1
2022-01-01 02:00:00     2
2022-01-01 03:00:00     3
2022-01-01 04:00:00     4
2022-01-01 05:00:00     5
2022-01-01 06:00:00     6
2022-01-01 07:00:00     7
2022-01-01 08:00:00     8
2022-01-01 09:00:00     9
2022-01-01 10:00:00    10
2022-01-01 11:00:00    11
Freq: H, dtype: int32

In [8]:
# 通过计算每一组的加和将这些数据聚合到5小时的块或柱内
ts.resample("5h").sum()

2022-01-01 00:00:00    10
2022-01-01 05:00:00    35
2022-01-01 10:00:00    21
Freq: 5H, dtype: int32

### closed参数：

In [9]:
# closed="left"：向下采样时每个区间的左端是封闭的(包含的)
# 此时，传递的频率以5小时的增量定义箱体边界。对于这个频率，默认情况下，左箱体边界是包含的，
# 因此00:00值包含在00:00 到05:00 时间间隔内，而05:00值不包含在该时间间隔内。
ts.resample("5h" , closed="left").sum()

2022-01-01 00:00:00    10
2022-01-01 05:00:00    35
2022-01-01 10:00:00    21
Freq: 5H, dtype: int32

In [10]:
# closed="right"：向下采样时每个区间的右端是封闭的(包含的)
# 因此00:00值不包含在00:00 到05:00 时间间隔内，而05:00值包含在该时间间隔内。
ts.resample("5h" , closed="right").sum()

2021-12-31 19:00:00     0
2022-01-01 00:00:00    15
2022-01-01 05:00:00    40
2022-01-01 10:00:00    11
Freq: 5H, dtype: int32

### label参数：

In [11]:
# 生成的时间序列由每个箱体左侧的时间戳标记。通过传递 label="right"，你可以用右箱体边界标记它们
ts.resample("5h" , closed="right" , label="right").sum()

2022-01-01 00:00:00     0
2022-01-01 05:00:00    15
2022-01-01 10:00:00    40
2022-01-01 15:00:00    11
Freq: 5H, dtype: int32

### to_offset：

In [12]:
from pandas.tseries.frequencies import to_offset

In [13]:
result = ts.resample("5h", closed="right", label="right").sum()
result

2022-01-01 00:00:00     0
2022-01-01 05:00:00    15
2022-01-01 10:00:00    40
2022-01-01 15:00:00    11
Freq: 5H, dtype: int32

In [14]:
result.index = result.index + to_offset("-1h")

In [15]:
result

2021-12-31 23:00:00     0
2022-01-01 04:00:00    15
2022-01-01 09:00:00    40
2022-01-01 14:00:00    11
Freq: 5H, dtype: int32

## 1.1 开端-峰值-谷值-结束（OHLC）重新采样：

### ohlc 聚合函数：

In [16]:
ts = pd.Series(np.random.permutation(np.arange(len(dates))), index=dates)
ts

2022-01-01 00:00:00     8
2022-01-01 01:00:00     7
2022-01-01 02:00:00    11
2022-01-01 03:00:00     1
2022-01-01 04:00:00     0
2022-01-01 05:00:00     4
2022-01-01 06:00:00     5
2022-01-01 07:00:00     6
2022-01-01 08:00:00     9
2022-01-01 09:00:00     3
2022-01-01 10:00:00    10
2022-01-01 11:00:00     2
Freq: H, dtype: int32

In [17]:
ts.resample("5h").ohlc()

,open,high,low,close
2022-01-01 00:00:00,8,11,0,0
2022-01-01 05:00:00,4,9,3,3
2022-01-01 10:00:00,10,10,2,2


# 2.向上采样（Upsampling ）和插值（Interpolation）：

In [18]:
# 首先生成一个包含每周数据的DataFrame
df = pd.DataFrame(np.arange(6).reshape((2, 3)),
                     index=pd.date_range("2022-01-01",periods=2,freq="W-MON"),# W-MON：每周一
                     columns=["A", "B", "C"])

df

,A,B,C
2022-01-03,0,1,2
2022-01-10,3,4,5


### 当对该数据使用聚合函数时，每一组只有一个值，会在间隙中产生缺失值。使用 asfreq 方法在不聚合的情况下转换到更高频率：

In [19]:
df_daily = df.resample("D").asfreq()
df_daily

,A,B,C
2022-01-03,0.0,1.0,2.0
2022-01-04,NaN,NaN,NaN
2022-01-05,NaN,NaN,NaN
2022-01-06,NaN,NaN,NaN
2022-01-07,NaN,NaN,NaN
2022-01-08,NaN,NaN,NaN
2022-01-09,NaN,NaN,NaN
2022-01-10,3.0,4.0,5.0


### fillna和reindex方法中可用的填充或插值方法可用于重采样。

In [20]:
# 假设你想在非周一的日期上向前填充每周数值：
df.resample("D").ffill()

,A,B,C
2022-01-03,0,1,2
2022-01-04,0,1,2
2022-01-05,0,1,2
2022-01-06,0,1,2
2022-01-07,0,1,2
2022-01-08,0,1,2
2022-01-09,0,1,2
2022-01-10,3,4,5


In [21]:
# 也可以选择仅向前填充一定数量的区间，以限制继续使用观测值的时距离
df.resample("D").ffill(limit=3)

,A,B,C
2022-01-03,0.0,1.0,2.0
2022-01-04,0.0,1.0,2.0
2022-01-05,0.0,1.0,2.0
2022-01-06,0.0,1.0,2.0
2022-01-07,NaN,NaN,NaN
2022-01-08,NaN,NaN,NaN
2022-01-09,NaN,NaN,NaN
2022-01-10,3.0,4.0,5.0


### 需要注意的是，新日期索引根本不需要与旧日期索引重合：

In [22]:
# W-TUE：每周二
df.resample("W-TUE").ffill()

,A,B,C
2022-01-04,0,1,2
2022-01-11,3,4,5
